## Addendum Simulatie_les4
In dit notebook wordt dieper in gegaan op de *problemen* die kunnen onstaan bij het aggregeren van DataFrames.

### Missing values
Ten eerste gaan we kijken naar een situatie waarbij er, zonder dat het direct te zien is, *missing values* ontstaan.

In [3]:
import pandas as pd

In [4]:
# Voorbeelddataset waarin de verkoopgegevens van een pizzeria verzameld zijn.
# Er zijn 3 dagen waarover in totaal 8 klanten pizza's besteld hebben. Sommige klanten hebben meerdere pizza's besteld. 
# De laatste kolom is het belangrijkste: hierin is aangegeven welke soort pizza besteld is.
df = pd.read_csv('../databronnen/pizza.csv', sep=';')
df

,Dag,Klant,Pizza
0,1,1,Margherita
1,1,2,Tonno
2,1,3,Margherita
3,1,3,Diavola
4,1,3,Tonno
5,2,4,Diavola
6,2,5,Margherita
7,2,5,Margherita
8,3,6,Tonno
9,3,7,Margherita


Het doel is om de verkoopcijfers van de verschillende soorten pizza's te analyseren in verband met het aankopen van ingrediënten. Hiervoor willen we voor elk soort pizza berekenen hoeveel er gemiddeld per dag verkocht worden. 

**Vraag:** *hoeveel pizza Margherita's, Diavola's en Tonno's worden er gemiddeld per dag verkocht?*

In [5]:
# Met behulp van onderstaande groupby tellen we voor elke dag hoeveel pizza's er van elk soort verkocht zijn.
df_agg = df.groupby(['Pizza', 'Dag']).agg({'Klant': 'count'})
df_agg

Klant
Pizza      Dag       
Diavola    1        1
           2        1
           3        1
Margherita 1        2
           2        2
           3        2
Tonno      1        2
           3        1

In [6]:
# Bovenstaand DataFrame is wat lastiger verder te gebruiken vanwege de kolomnamen, met .reset_index() sla je het DataFrame plat en kun je het makkelijker gebruiken
df_agg = df.groupby(['Pizza', 'Dag']).agg({'Klant': 'count'}).reset_index()
df_agg

,Pizza,Dag,Klant
0,Diavola,1,1
1,Diavola,2,1
2,Diavola,3,1
3,Margherita,1,2
4,Margherita,2,2
5,Margherita,3,2
6,Tonno,1,2
7,Tonno,3,1


In [7]:
# Je kunt bijvoorbeeld het gemiddelde aantal pizza's per dag berekenen
df_agg.groupby('Pizza').agg({'Klant': 'mean'})

,Klant
Pizza,
Diavola,1.0
Margherita,2.0
Tonno,1.5


**Vraag:** *wat gaat hier mis? Het gemiddeld aantal Tonno's klopt niet.*

Bovenstaande berekening klopt niet want df_agg mist een rij voor ```Tonno``` op dag 2: toen waren er 0 aankopen. Hierdoor is het gemiddelde niet 1 maar 1.5 geworden.

Het probleem dat we moeten oplossen is dat bij de ```.agg({'Klant': 'count'})``` resultaten met waarde 0 **niet** meegenomen worden maar **wel** belangrijk zijn.

Er zijn verschillende methodes om dit probleem op te lossen. 

In [8]:
# De unstack functie gebruiken is een optie wanneer je weinig verschillende soorten pizza's hebt
df_agg = df.groupby(['Dag', 'Pizza']).agg({'Pizza': 'count'}).unstack()
df_agg

Pizza                 
Pizza Diavola Margherita Tonno
Dag                           
1         1.0        2.0   2.0
2         1.0        2.0   NaN
3         1.0        2.0   1.0

In [9]:
# unstack heeft een optie om lege waardes te vullen
df_agg = df.groupby(['Dag', 'Pizza']).agg({'Pizza': 'count'}).unstack(fill_value=0)
df_agg

Pizza                 
Pizza Diavola Margherita Tonno
Dag                           
1           1          2     2
2           1          2     0
3           1          2     1

Maar bovenstaand DataFrame is niet zo handig wanneer je het daarna wilt gebruiken voor visualisaties, zeker niet wanneer er veel verschillende kolommen zijn. Werken met een *compleet* DataFrame is handiger.

In [10]:
# In onderstaand codeblok wordt het DataFrame op een manier opgevuld dat het resultaat een handiger DataFrame is 
import itertools

# Alle unieke waardes voor soorten pizza's en dagen bepalen
pizzas = df['Pizza'].unique()
dagen = df['Dag'].unique()

# Alle combinaties van deze soorten pizza's en dagen bepalen zodat er een compleet DataFrame gemaakt kan worden
all_combos = pd.DataFrame(list(itertools.product(pizzas, dagen)), columns=['Pizza', 'Dag'])
all_combos


,Pizza,Dag
0,Margherita,1
1,Margherita,2
2,Margherita,3
3,Tonno,1
4,Tonno,2
5,Tonno,3
6,Diavola,1
7,Diavola,2
8,Diavola,3


In [11]:
# De groupby die voor incomplete data zorgt
df_agg = df.groupby(['Pizza', 'Dag']).agg({'Klant': 'count'}).reset_index()

# all_combos is leeg maar bevat alle combinaties
# df_agg is gevuld maar bevat niet alle combinaties
# Door ze te combineren met een left join, komen alle waardes uit df_agg in all_combos
# Het is belangrijk dat de missende waardes uit df_agg opgevuld worden met 0
df_agg_full = all_combos.merge(df_agg, on=['Pizza', 'Dag'], how='left').fillna({'Klant': 0})

df_agg_full

,Pizza,Dag,Klant
0,Margherita,1,2.0
1,Margherita,2,2.0
2,Margherita,3,2.0
3,Tonno,1,2.0
4,Tonno,2,0.0
5,Tonno,3,1.0
6,Diavola,1,1.0
7,Diavola,2,1.0
8,Diavola,3,1.0


In [12]:
# De berekening klopt nu wel
df_agg_full.groupby('Pizza').agg({'Klant': 'mean'})

,Klant
Pizza,
Diavola,1.0
Margherita,2.0
Tonno,1.0


#### Plat slaan
Daarnaast ontstaan er soms complexe DataFrames wanneer er meerdere aggregaties plaats vinden.

In [13]:
# Stel dat we weer naar df_agg_full kijken
df_agg_full


,Pizza,Dag,Klant
0,Margherita,1,2.0
1,Margherita,2,2.0
2,Margherita,3,2.0
3,Tonno,1,2.0
4,Tonno,2,0.0
5,Tonno,3,1.0
6,Diavola,1,1.0
7,Diavola,2,1.0
8,Diavola,3,1.0


In [14]:
# Maar deze keer berekenen we zowel de som van het aantal pizza's over alle dagen als het gemiddelde per dag
# Het is dus een aggregatie op een geaggregeerd DataFrame, vandaar de lange naam...
df_agg_full_agg = df_agg_full.groupby('Pizza').agg({'Klant': ['sum', 'mean']})
df_agg_full_agg

Klant     
             sum mean
Pizza                
Diavola      3.0  1.0
Margherita   6.0  2.0
Tonno        3.0  1.0

In [15]:
# De kolommen aanroepen is lastig, kijk maar wat de namen zijn
df_agg_full_agg.columns

MultiIndex([('Klant',  'sum'),
            ('Klant', 'mean')],
           )

In [16]:
# Het kan wel en het kan voordelen hebben wanneer je gaat loopen over de originele kolomnamen en de aggregaties
df_agg_full_agg[('Klant', 'sum')]

Pizza
Diavola       3.0
Margherita    6.0
Tonno         3.0
Name: (Klant, sum), dtype: float64

In [17]:
# Maar je kunt de kolomnamen ook plat slaan
# Onderstaande code neemt een '_' als basis en join de beide waardes uit het tuple van elke kolomnaam en stript eventuele spaties
df_agg_full_agg.columns = ['_'.join(col).strip() for col in df_agg_full_agg.columns.values]
df_agg_full_agg


,Klant_sum,Klant_mean
Pizza,,
Diavola,3.0,1.0
Margherita,6.0,2.0
Tonno,3.0,1.0


In [18]:
# Het nieuwe DataFrame heeft kolomnamen die je makkelijker kunt aanroepen.
df_agg_full_agg['Klant_sum']


Pizza
Diavola       3.0
Margherita    6.0
Tonno         3.0
Name: Klant_sum, dtype: float64